In [102]:
import numpy as np
import pandas as pd

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Joining the data

First of all, since we have three datasets, let's join them all into one big dataset.

In [103]:
data_houses = pd.read_csv("dataset_houses.csv")
data_apartments = pd.read_csv("dataset_apartments.csv")
data_sharehouses = pd.read_csv("dataset_sharehouses.csv")

data = pd.concat([data_houses, data_apartments, data_sharehouses])

## Exploratory analysis

We will now do a quick exploratory data analysis.

In [104]:
# Describe the numeric attributes.
data.describe()

,Rooms,Bedrooms,Number of floors,Built in year,Register number
count,7597.000000,1663.000000,2663.000000,6622.000000,8.900000e+02
mean,3.350401,3.265183,1.885468,1986.282845,8.544396e+06
std,2.342659,1.794841,0.902053,61.279443,1.732335e+07
min,1.000000,1.000000,1.000000,1.000000,1.600000e+01
25%,2.000000,3.000000,1.000000,1969.000000,1.630701e+06
50%,3.000000,3.000000,2.000000,1995.000000,3.047403e+06
75%,4.000000,4.000000,2.000000,2021.000000,9.874400e+06
max,47.000000,33.000000,19.000000,2105.000000,1.208426e+08


In [105]:
data.describe(include=[object])

,Title,Description,Link,Location,Total area,Condition,Readiness,Ownership,Energy mark,Data from realestate book,...,Kitchen,Sanitary arrangements,Heating and ventilation,Communications and security,This floor/Number of floors,Neighbourhood,Ground area,Kulud suvel/talvel,Cadastre no.,Additional information
count,7988,7988,7988,7816,7837,7068,1186,6993,5949,4498,...,5052,6149,6357,5373,5083,4332,2102,471,4165,709
unique,6570,2635,7988,5663,2038,8,5,7,10,1,...,423,1319,614,605,168,1737,1577,242,2994,596
top,"Apartment for sale, 2 rooms, Järve 2, Kristiin...","Apartment ownership, stone house",https://www.kv.ee/muua-tallinna-lahedal-eksklu...,"59.4019286,24.7271115",120 m²,all brand-new,ready,apartment ownership,Missing,Data from realestate book,...,open kitchen,shower,central heating,frontdoor locked,1/2,roads paved roads,600 m²,60 € / 120 €,79301:001:0984,"underground garage, box-room, new electricity,..."
freq,31,2203,1,60,51,2499,1047,4753,1697,4498,...,745,412,1983,464,477,272,24,16,36,20


In [106]:
data.columns

Index(['Title', 'Description', 'Link', 'Location', 'Rooms', 'Bedrooms',
       'Total area', 'Number of floors', 'Built in year', 'Condition',
       'Readiness', 'Ownership', 'Energy mark', 'Data from realestate book',
       'Notify about incorrect advertisement', 'Lisainfo', 'Kitchen',
       'Sanitary arrangements', 'Heating and ventilation',
       'Communications and security', 'This floor/Number of floors',
       'Neighbourhood', 'Ground area', 'Kulud suvel/talvel', 'Cadastre no.',
       'Register number', 'Additional information'],
      dtype='object')

## Cleaning the data

First, drop the columns which are useless for predicting the prices. Apart from the obvious ones, drop also the summer/winter costs because there are so few samples.

In [107]:
data.drop(columns=["Title", "Description", "Link", "Cadastre no.", "Data from realestate book", "Notify about incorrect advertisement", "Kulud suvel/talvel"], inplace=True)

data.describe(include=[object])
data.columns

,Location,Total area,Condition,Readiness,Ownership,Energy mark,Lisainfo,Kitchen,Sanitary arrangements,Heating and ventilation,Communications and security,This floor/Number of floors,Neighbourhood,Ground area,Additional information
count,7816,7837,7068,1186,6993,5949,6330,5052,6149,6357,5373,5083,4332,2102,709
unique,5663,2038,8,5,7,10,4859,423,1319,614,605,168,1737,1577,596
top,"59.4019286,24.7271115",120 m²,all brand-new,ready,apartment ownership,Missing,"ventilation, parquet, lift, parking free parking",open kitchen,shower,central heating,frontdoor locked,1/2,roads paved roads,600 m²,"underground garage, box-room, new electricity,..."
freq,60,51,2499,1047,4753,1697,82,745,412,1983,464,477,272,24,20


Index(['Location', 'Rooms', 'Bedrooms', 'Total area', 'Number of floors',
       'Built in year', 'Condition', 'Readiness', 'Ownership', 'Energy mark',
       'Lisainfo', 'Kitchen', 'Sanitary arrangements',
       'Heating and ventilation', 'Communications and security',
       'This floor/Number of floors', 'Neighbourhood', 'Ground area',
       'Register number', 'Additional information'],
      dtype='object')

## Simple data type conversions

Remove the m2 from the area field to convert them to numeric.

In [108]:
data["Total area"] = data['Total area'].astype('str').map(lambda x: x.split()[0]).astype('float64') #remove m2
data["Ground area"] = data['Ground area'].astype('str').map(lambda x: x.split()[0]).astype('float64') #remove m2

Convert the "Energy mark" into a number.

In [109]:
def remap_column(data, column, mapping):
    data[column] = data[column].map(mapping)

In [110]:
data["Energy mark"].unique()
# data["Energy mark"] = data["Energy mark"].map({'C': 2, 'B': 1, np.nan: 0, '-': 0, 'Missing': 0, 'A': 0, 'H': 7, 'E': 4, 'D': 3, 'G': 6, 'F': 5})
remap_column(data, "Energy mark", {'C': 2, 'B': 1, np.nan: 0, '-': 0, 'Missing': 0, 'A': 0, 'H': 7, 'E': 4, 'D': 3, 'G': 6, 'F': 5})
data["Energy mark"].unique()

array(['C', 'B', nan, '-', 'Missing', 'A', 'H', 'E', 'D', 'G', 'F'],
      dtype=object)

array([2, 1, 0, 7, 4, 3, 6, 5], dtype=int64)

In [111]:
data["Condition"].unique()
remap_column(data, "Condition", {'all brand-new': 6, 'Good condition': 5, 'ready': 5, 'needs renovating': 0, 'satisfactory': 1, np.nan: 1, 'sanitary renovation needed': 2, 'renovated': 4,'sanitary renovation done': 3})
data["Condition"].unique()

array(['all brand-new', 'Good condition', 'ready', 'needs renovating',
       'satisfactory', nan, 'sanitary renovation needed', 'renovated',
       'sanitary renovation done'], dtype=object)

array([6, 5, 0, 1, 2, 4, 3], dtype=int64)

In [112]:
data["Readiness"].unique()
remap_column(data, "Readiness", {'ready': 4, 'roofed box': 2, np.nan: 4, 'box with doors and windows': 3, 'roofless box': 1, 'foundation': 0})
data["Readiness"].unique()

array(['ready', 'roofed box', nan, 'box with doors and windows',
       'roofless box', 'foundation'], dtype=object)

array([4, 2, 3, 1, 0], dtype=int64)

In [113]:
data["Ownership"].unique()

array(['private property', nan, 'apartment association',
       'joint ownership', 'apartment ownership', 'movable',
       'building lease', 'logical part'], dtype=object)

## Separating features

First of all, try to find which columns contain many different features.

Before converting columns to multiple features, it might be useful to assign reasonable values to NaNs in those columns!

In [114]:
# Create a function to assist in removing invalid values.
def fill_na_with_mode(data, column):
    return data[column].fillna(value=data[column].mode().values[0], inplace=True)

def fill_na_with_mean(data, column):
    return data[column].fillna(value=data[column].mean(), inplace=True)

In [115]:
fill_na_with_mode(data, "Ownership")
data = pd.get_dummies(data, columns=["Ownership"])

Now, some more complex feature extraction.

First, what are the unique feature values in the column "Lisainfo".

In [116]:
unique_features = set()

for entry in data["Lisainfo"].unique():
    if not pd.isna(entry):
        features = entry.strip().split(", ")
        #print(features)
        unique_features.update(features)

unique_features

{'3*380V',
 'TV-set',
 'balcony',
 'balcony ',
 'balcony 0.5 m²',
 'balcony 0.9 m²',
 'balcony 1 m²',
 'balcony 1.4 m²',
 'balcony 1.5 m²',
 'balcony 1.6 m²',
 'balcony 1.8 m²',
 'balcony 10 m²',
 'balcony 10.0 m²',
 'balcony 10.1 m²',
 'balcony 10.2 m²',
 'balcony 10.3 m²',
 'balcony 10.4 m²',
 'balcony 10.5 m²',
 'balcony 10.6 m²',
 'balcony 10.7 m²',
 'balcony 10.8 m²',
 'balcony 10.9 m²',
 'balcony 11 m²',
 'balcony 11.2 m²',
 'balcony 11.4 m²',
 'balcony 11.5 m²',
 'balcony 11.6 m²',
 'balcony 11.7 m²',
 'balcony 11.8 m²',
 'balcony 11.9 m²',
 'balcony 12 m²',
 'balcony 12.1 m²',
 'balcony 12.2 m²',
 'balcony 12.3 m²',
 'balcony 12.4 m²',
 'balcony 12.5 m²',
 'balcony 12.6 m²',
 'balcony 12.7 m²',
 'balcony 12.8 m²',
 'balcony 12.9 m²',
 'balcony 13 m²',
 'balcony 13.1 m²',
 'balcony 13.2 m²',
 'balcony 13.3 m²',
 'balcony 13.4 m²',
 'balcony 13.6 m²',
 'balcony 13.8 m²',
 'balcony 13.9 m²',
 'balcony 14 m²',
 'balcony 14.1 m²',
 'balcony 14.2 m²',
 'balcony 14.4 m²',
 'balcony 14

Extract numeric features

In [144]:
data_balcony = []
data_balcony_size = []

for i in data["Lisainfo"].values:
    balcony = 0 #Base values
    balcony_size = 0.0
    
    if not pd.isna(i):
        features = i.strip().split(", ")
        
        for feature in features:
            if feature.find("balcony") == 0:
                balcony = 1
                balcony_el = feature.strip().split()
                if len(balcony_el) > 1:
                    balcony_size = float(balcony_el[1])
                    
    data_balcony.append(balcony)
    data_balcony_size.append(balcony_size)

data["Balcony"] = data_balcony
data["Balcony size"] = data_balcony_size

In [145]:
#Probably don't care about that feature
data_dist_from_Tallinn = []

for i in data["Lisainfo"].values:
    dist_from_Tallinn = 0.0
    
    if not pd.isna(i):
        features = i.strip().split(", ")
        
        for feature in features:
            if feature.find("distance from Tallinn") == 0:
                dist_from_Tallinn = float(feature.strip().split()[3])
    data_dist_from_Tallinn.append(dist_from_Tallinn)

data["Distance from Tallinn"] = data_dist_from_Tallinn

Extract categorical features

In [122]:
data_addition = []

for i in data["Lisainfo"].values:
    current = np.nan

    if not pd.isna(i):
        features = i.strip().split(", ")

        for feature in features:
            if feature.find("parking") == 0:
                current = ' '.join(feature.strip().split()[1:-1])
    data_addition.append(current)

data["Parking"] = data_addition

In [124]:
data_addition = []

for i in data["Lisainfo"].values:
    current = np.nan

    if not pd.isna(i):
        features = i.strip().split(", ")

        for feature in features:
            if feature.find("wall") == 0:
                current = ' '.join(feature.strip().split()[1:])
    data_addition.append(current)

data["Wall"] = data_addition

In [127]:
data_addition = []

for i in data["Lisainfo"].values:
    current = np.nan

    if not pd.isna(i):
        features = i.strip().split(", ")

        for feature in features:
            if feature.find("roof") == 0:
                current = ' '.join(feature.strip().split()[1:-1])
    data_addition.append(current)

data["Roof"] = data_addition

Remove the already separated features from 'unique features'

In [131]:
filtered_unique_features = [ x for x in unique_features if "balcony" not in x and 
                                                            "distance from Tallinn" not in x and 
                                                            "parking" not in x and
                                                            "wall" not in x and 
                                                            "roof" not in x ]

Separate the remaining features

In [141]:
def separate(dataset, uniques):
    for unique_feature in uniques:
        data_addition = []

        for i in data["Lisainfo"].values:
            current = False

            if not pd.isna(i):
                features = i.strip().split(", ")

                for feature in features:
                    if feature.find(unique_feature) == 0:
                        current = True
            data_addition.append(current)

        dataset[unique_feature] = data_addition
        
separate(data, filtered_unique_features)

## Dealing with NaN values

In [79]:
# data.dtypes

data["Ownership"].mode().values[0]

for column in data.columns:
    dtype = data[column].dtype

    if dtype == np.object_:
        print(data[column].mode())
        data[column].fillna(value=data[column].mode().values[0], inplace=True)
    else:
        data[column].fillna(value=data[column].mean(), inplace=True)

'apartment ownership'

0    59.4019286,24.7271115
dtype: object
0    all brand-new
dtype: object
0    ready
dtype: object
0    apartment ownership
dtype: object
0    ventilation, parquet, lift, parking free parking
dtype: object
0    open kitchen
dtype: object
0    shower
dtype: object
0    central heating
dtype: object
0    frontdoor locked
dtype: object
0    1/2
dtype: object
0    roads paved roads
dtype: object
0    60 € /  120 €
dtype: object
0    underground garage, box-room, new electricity,...
dtype: object


In [80]:
data.describe(include=[np.object])

C:\Users\Jyri\AppData\Local\Temp/ipykernel_20272/1422700444.py:1: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  data.describe(include=[np.object])


,Location,Condition,Readiness,Ownership,Lisainfo,Kitchen,Sanitary arrangements,Heating and ventilation,Communications and security,This floor/Number of floors,Neighbourhood,Kulud suvel/talvel,Additional information
count,7988,7988,7988,7988,7988,7988,7988,7988,7988,7988,7988,7988,7988
unique,5663,8,5,7,4859,423,1319,614,605,168,1737,242,596
top,"59.4019286,24.7271115",all brand-new,ready,apartment ownership,"ventilation, parquet, lift, parking free parking",open kitchen,shower,central heating,frontdoor locked,1/2,roads paved roads,60 € / 120 €,"underground garage, box-room, new electricity,..."
freq,232,3419,7849,5748,1740,3681,2251,3614,3079,3382,3928,7533,7299


In [81]:
data.isna()

,Location,Rooms,Bedrooms,Total area,Number of floors,Built in year,Condition,Readiness,Ownership,Energy mark,...,Kitchen,Sanitary arrangements,Heating and ventilation,Communications and security,This floor/Number of floors,Neighbourhood,Ground area,Kulud suvel/talvel,Register number,Additional information
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
504,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
505,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
506,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
507,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
